# Fuga de datos (Data Leakage)
Encuentre y solucione este problema que arruina su modelo de manera sutil.

En este tutorial, aprenderá qué es la fuga de datos y cómo prevenirla. Si no sabe cómo prevenirlo, aparecerán fugas con frecuencia y arruinarán sus modelos de maneras sutiles y peligrosas. Entonces, este es uno de los conceptos más importantes para los científicos de datos en ejercicio.

## Introducción

La fuga de datos (o fuga) ocurre cuando sus datos de entrenamiento contienen información sobre el objetivo, pero datos similares no estarán disponibles cuando el modelo se use para la predicción. Esto conduce a un alto rendimiento en el conjunto de entrenamiento (y posiblemente incluso en los datos de validación), pero el modelo tendrá un rendimiento deficiente en producción.

En otras palabras, la fuga hace que un modelo parezca preciso hasta que se empieza a tomar decisiones con el modelo, y luego el modelo se vuelve muy inexacto.

Hay dos tipos principales de fugas: fugas objetivo y contaminación de prueba de tren.

## Fuga objetivo

La fuga de objetivos se produce cuando sus predictores incluyen datos que no estarán disponibles en el momento de realizar las predicciones. Es importante pensar en la fuga de objetivos en términos del momento o el orden cronológico en que los datos están disponibles, no simplemente si una característica ayuda a hacer buenas predicciones.

Un ejemplo será útil. Imagine que quiere predecir quién enfermará de neumonía. Las primeras filas de sus datos sin procesar se ven así:

got_pneumonia	age 	 weight 	male	took_antibiotic_medicine	...

False       	65  	100     	False	False	...

False       	72  	130     	True	False	...

True        	58  	100     	False	True	...

Las personas toman antibióticos después de contraer neumonía para recuperarse. Los datos sin procesar muestran una fuerte relación entre esas columnas, pero take_antibiotic_medicine se cambia con frecuencia después de que se determina el valor de got_pneumonia. Esta es una fuga de objetivo.

El modelo vería que cualquiera que tenga un valor Falso para tomó_antibiótico_medicina no tenía neumonía. Dado que los datos de validación provienen de la misma fuente que los datos de entrenamiento, el patrón se repetirá en la validación y el modelo tendrá excelentes puntuaciones de validación (o validación cruzada).

Pero el modelo será muy inexacto cuando se implemente posteriormente en el mundo real, porque incluso los pacientes que contraerán neumonía aún no habrán recibido antibióticos cuando necesitemos hacer predicciones sobre su salud futura.

Para evitar este tipo de fuga de datos, se debe excluir cualquier variable actualizada (o creada) después de alcanzar el valor objetivo.

## Contaminación de prueba de tren
Se produce un tipo diferente de fuga cuando no se tiene cuidado de distinguir los datos de entrenamiento de los datos de validación.

Recuerde que la validación pretende ser una medida de cómo funciona el modelo con datos que no ha considerado antes. Puede corromper este proceso de manera sutil si los datos de validación afectan el comportamiento de preprocesamiento. A esto a veces se le llama contaminación de prueba de tren.

Por ejemplo, imagina que ejecutas un preprocesamiento (como ajustar un imputador para los valores faltantes) antes de llamar a train_test_split(). Su modelo puede obtener buenas puntuaciones de validación, lo que le dará una gran confianza en él, pero tendrá un rendimiento deficiente cuando lo implemente para tomar decisiones.

Después de todo, usted incorporó datos de la validación o los datos de prueba en la forma en que hace predicciones, por lo que pueden funcionar bien con esos datos en particular, incluso si no se pueden generalizar a datos nuevos. Este problema se vuelve aún más sutil (y más peligroso) cuando realiza ingeniería de funciones más compleja.

Si su validación se basa en una división simple de prueba de tren, excluya los datos de validación de cualquier tipo de ajuste, incluido el ajuste de pasos de preprocesamiento. Esto es más fácil si utiliza canalizaciones de scikit-learn. Cuando se utiliza validación cruzada, es aún más importante que realice el preprocesamiento dentro del proceso.

### Ejemplo
En este ejemplo, aprenderá una forma de detectar y eliminar fugas de objetivos.

Usaremos un conjunto de datos sobre solicitudes de tarjetas de crédito y omitiremos el código de configuración de datos básicos. El resultado final es que la información sobre cada solicitud de tarjeta de crédito se almacena en un DataFrame X. Lo usaremos para predecir qué solicitudes fueron aceptadas en una Serie y.

In [1]:
import pandas as pd

# Read the data
data = pd.read_csv('/kaggle/input/aer-credit-card-data/AER_credit_card_data.csv', 
                   true_values = ['yes'], false_values = ['no'])

# Select target
y = data.card

# Select predictors
X = data.drop(['card'], axis=1)

print("Number of rows in the dataset:", X.shape[0])
X.head()

Number of rows in the dataset: 1319


,reports,age,income,share,expenditure,owner,selfemp,dependents,months,majorcards,active
0,0,37.66667,4.5200,0.033270,124.983300,True,False,3,54,1,12
1,0,33.25000,2.4200,0.005217,9.854167,False,False,3,34,1,13
2,0,33.66667,4.5000,0.004156,15.000000,True,False,4,58,1,5
3,0,30.50000,2.5400,0.065214,137.869200,False,False,0,25,1,7
4,0,32.16667,9.7867,0.067051,546.503300,True,False,2,64,1,5


Dado que se trata de un conjunto de datos pequeño, utilizaremos validación cruzada para garantizar medidas precisas de la calidad del modelo.

In [3]:
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Since there is no preprocessing, we don't need a pipeline (used anyway as best practice!)
my_pipeline = make_pipeline(RandomForestClassifier(n_estimators=100))
cv_scores = cross_val_score(my_pipeline, X, y, 
                            cv=5,
                            scoring='accuracy')

print("Cross-validation accuracy: %f" % cv_scores.mean())

Cross-validation accuracy: 0.981052


Con experiencia, descubrirá que es muy raro encontrar modelos que sean precisos el 98% de las veces. Sucede, pero es bastante poco común como para que debamos inspeccionar los datos más de cerca para detectar fugas de objetivos.

Aquí hay un resumen de los datos, que también puede encontrar en la pestaña de datos:

* tarjeta: 1 si se acepta la solicitud de tarjeta de crédito, 0 si no

* informes: Número de informes despectivos importantes

* edad: Edad n años más doceavos de año

* Ingresos: Ingresos anuales (divididos por 10,000)

* Participación: Relación entre el gasto mensual en tarjetas de crédito y los ingresos anuales.

* gasto: Gasto promedio mensual con tarjeta de crédito

* propietario: 1 si es propietario de la casa, 0 si alquila

* selfempl: 1 si trabaja por cuenta propia, 0 si no

* dependientes: 1 + número de dependientes

* meses: meses viviendo en la dirección actual

* majorcards: Número de tarjetas de crédito principales poseídas

* activo: Número de cuentas de crédito activas

 Algunas variables parecen sospechosas. Por ejemplo, ¿gasto significa gasto en esta tarjeta o en tarjetas utilizadas antes de presentar la solicitud?

En este punto, las comparaciones de datos básicos pueden resultar muy útiles:

In [4]:
expenditures_cardholders = X.expenditure[y]
expenditures_noncardholders = X.expenditure[~y]

print('Fraction of those who did not receive a card and had no expenditures: %.2f' \
      %((expenditures_noncardholders == 0).mean()))
print('Fraction of those who received a card and had no expenditures: %.2f' \
      %(( expenditures_cardholders == 0).mean()))

Fraction of those who did not receive a card and had no expenditures: 1.00
Fraction of those who received a card and had no expenditures: 0.02


Como se muestra arriba, todos los que no recibieron una tarjeta no tuvieron gastos, mientras que sólo el 2% de los que recibieron una tarjeta no tuvieron gastos. No es sorprendente que nuestro modelo pareciera tener una alta precisión. Pero este también parece ser un caso de fuga de objetivos, donde los gastos probablemente significan gastos en la tarjeta que solicitaron.

Dado que la participación está parcialmente determinada por el gasto, también debería excluirse. Las variables activas y cartas principales son un poco menos claras, pero por la descripción suenan preocupantes. En la mayoría de las situaciones, es mejor prevenir que curar si no puedes localizar a las personas que crearon los datos para obtener más información.

Ejecutaríamos un modelo sin fuga objetivo de la siguiente manera:

In [5]:
# Drop leaky predictors from dataset
potential_leaks = ['expenditure', 'share', 'active', 'majorcards']
X2 = X.drop(potential_leaks, axis=1)

# Evaluate the model with leaky predictors removed
cv_scores = cross_val_score(my_pipeline, X2, y, 
                            cv=5,
                            scoring='accuracy')

print("Cross-val accuracy: %f" % cv_scores.mean())

Cross-val accuracy: 0.833198


Esta precisión es bastante menor, lo que puede resultar decepcionante. Sin embargo, podemos esperar que sea correcto aproximadamente el 80% de las veces cuando se use en aplicaciones nuevas, mientras que el modelo con fugas probablemente funcionaría mucho peor que eso (a pesar de su puntuación aparente más alta en la validación cruzada).

### Conclusión

La fuga de datos puede suponer un error multimillonario en muchas aplicaciones de ciencia de datos. Una separación cuidadosa de los datos de capacitación y validación puede prevenir la contaminación de las pruebas de trenes, y las tuberías pueden ayudar a implementar esta separación. Asimismo, una combinación de precaución, sentido común y exploración de datos puede ayudar a identificar las fugas objetivo.